# Testing out Linear Programming

In [6]:
# install this if u haven't

# !pip install pulp

In [7]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, COIN_CMD

In [12]:
path = "data/parts_cleaned/"

In [15]:
# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data.csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data.csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("data/games_cleaned/top100.csv")

# Convert numeric columns from string to numeric types
cpus = cpus.apply(pd.to_numeric, errors='ignore')
gpus = gpus.apply(pd.to_numeric, errors='ignore')
rams = rams.apply(pd.to_numeric, errors='ignore')
cases = cases.apply(pd.to_numeric, errors='ignore')
motherboards = motherboards.apply(pd.to_numeric, errors='ignore')
psus = psus.apply(pd.to_numeric, errors='ignore')
storage = storage.apply(pd.to_numeric, errors='ignore')
games = games.apply(pd.to_numeric, errors='ignore')

C:\Users\berna\AppData\Local\Temp\ipykernel_16564\1985277100.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  cpus = cpus.apply(pd.to_numeric, errors='ignore')
C:\Users\berna\AppData\Local\Temp\ipykernel_16564\1985277100.py:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  gpus = gpus.apply(pd.to_numeric, errors='ignore')
C:\Users\berna\AppData\Local\Temp\ipykernel_16564\1985277100.py:16: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  rams = rams.apply(pd.to_numeric, errors='ignore')
C:\Users\berna\AppData\Local\Temp\ipykernel_16564\1985277100.py:17: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_n

In [16]:
rams.head()

,Brand,Model,Capacity (GB),Speed (MHz),Price
0,Kingston,HyperX FURY,4,2666,47
1,Kingston,HyperX FURY,8,2666,79
2,Kingston,HyperX FURY,8,3200,82
3,Kingston,HyperX FURY,16,2666,121
4,Kingston,HyperX FURY,16,3200,122


In [17]:
# this code block optimizies performance within budget 


# Select the game (copy a game name from top100.csv)
selected_game = 'Counter-Strike 2'
game_data = games[games['name'] == selected_game].iloc[0]

# Define budget
budget = 2500

# Create the LP problem
problem = LpProblem("Desktop_Optimization", LpMaximize)

# Define variables
cpu_vars = [LpVariable(f"cpu_{i}", cat="Binary") for i in range(len(cpus))]
gpu_vars = [LpVariable(f"gpu_{i}", cat="Binary") for i in range(len(gpus))]
ram_vars = [LpVariable(f"ram_{i}", lowBound=0, upBound=2, cat="Integer") for i in range(len(rams))]  # Variable for selecting 1 or 2 identical RAM modules
psu_vars = [LpVariable(f"psu_{i}", cat="Binary") for i in range(len(psus))]

# Objective function: Minimize cost
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus))),
    "Total_Cost",
)

# Budget constraint
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus)))
    <= budget,
    "Budget_Constraint",
)

# Performance benchmarks for CPU and GPU
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Score"] for i in range(len(cpus))) >= game_data['CPU'],
    "CPU_Performance_Constraint",
)
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Score"] for i in range(len(gpus))) >= game_data['GPU'],
    "GPU_Performance_Constraint",
)

# RAM Capacity Constraint (ensuring it has enough capacity for the game)
# Mutliply by 1024 because capacity is in GB no MB in RAM_Data.csv
problem += (
    lpSum(ram_vars[i] * rams.iloc[i]["Capacity (GB)"] * 1024 for i in range(len(rams))) >= game_data['memory'],  
    "RAM_Capacity_Constraint",
)

# Power constraint: Ensure PSU meets the recommended power for the selected GPU
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Recommended Power"] for i in range(len(gpus))) <= 
    lpSum(psu_vars[i] * psus.iloc[i]["Wattage"] for i in range(len(psus))),
    "PSU_Power_Constraint",
)

# Ensure at most 2 identical RAM are picked
problem += lpSum(ram_vars) >= 1, "Min_One_RAM"
problem += lpSum(ram_vars) <= 2, "Max_Two_RAM"

# Only one CPU, GPU, and PSU can be selected
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(ram_vars) == 1, "Select_One_RAM"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"

# Solve the problem
problem.solve()

# Output the results
selected_cpu = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Name"].values[0]
selected_gpu = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Name"].values[0]
selected_psu = psus.iloc[[i for i in range(len(psus)) if psu_vars[i].value() == 1]]["Name"].values[0]

# Find the RAM model where value > 0 (either 1 or 2 copies)
selected_ram_model_index = [i for i in range(len(rams)) if ram_vars[i].value() > 0]
if selected_ram_model_index:
    selected_ram_model = rams.iloc[selected_ram_model_index[0]]["Model"]
    selected_ram_count = int(ram_vars[selected_ram_model_index[0]].value())
else:
    selected_ram_model = None
    selected_ram_count = 0

# Get the benchmarks for selected CPU and GPU
selected_cpu_score = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Score"].values[0]
selected_gpu_score = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Score"].values[0]


# Calculate total cost
total_cost = (
    sum(cpu_vars[i].value() * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    sum(gpu_vars[i].value() * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    sum(ram_vars[i].value() * rams.iloc[i]["Price"] for i in range(len(rams))) +
    sum(psu_vars[i].value() * psus.iloc[i]["Price"] for i in range(len(psus)))
)

# Calculate the total RAM capacity
total_ram_capacity = selected_ram_count * rams.iloc[selected_ram_model_index[0]]["Capacity (GB)"] * 1024

# Display the results
print(f"Required CPU Benchmark for {selected_game}: {game_data['CPU']}")
print(f"Required GPU Benchmark for {selected_game}: {game_data['GPU']}")
print(f"Required Memory for {selected_game}: {game_data['memory']} MB\n")

print(f"Selected CPU: {selected_cpu}")
print(f"CPU Benchmark Score: {selected_cpu_score}\n")
print(f"Selected GPU: {selected_gpu}")
print(f"GPU Benchmark Score: {selected_gpu_score}\n")

# Display the selected RAM model and its count (either 1 or 2 copies)
print(f"Selected RAM Model: {selected_ram_model}")
print(f"Selected RAM Count: {selected_ram_count}")
print(f"Total RAM Capacity: {total_ram_capacity} MB\n")
print(f"Selected PSU: {selected_psu}")
print(f"Total Cost: {total_cost}")

TypeError: must be real number, not str

In [ ]:
# This code block gives the required parts for the game, and does not try to improve the recommended parts above the requirments given

# Select the game (copy a game name from top100.csv)
selected_game = 'Counter-Strike 2'
game_data = games[games['name'] == selected_game].iloc[0]

# Define budget
budget = 1500

# Create the LP problem
problem = LpProblem("Desktop_Optimization", LpMinimize)

# Define variables
cpu_vars = [LpVariable(f"cpu_{i}", cat="Binary") for i in range(len(cpus))]
gpu_vars = [LpVariable(f"gpu_{i}", cat="Binary") for i in range(len(gpus))]
ram_vars = [LpVariable(f"ram_{i}", lowBound=0, upBound=2, cat="Integer") for i in range(len(rams))]  # Variable for selecting 1 or 2 identical RAM modules
psu_vars = [LpVariable(f"psu_{i}", cat="Binary") for i in range(len(psus))]

# Objective function: Minimize total cost
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus))),
    "Total_Cost",
)

# Budget constraint
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    lpSum(gpu_vars[i] * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    lpSum(ram_vars[i] * rams.iloc[i]["Price"] for i in range(len(rams))) +
    lpSum(psu_vars[i] * psus.iloc[i]["Price"] for i in range(len(psus)))
    <= budget,
    "Budget_Constraint",
)

# Performance benchmarks for CPU and GPU
problem += (
    lpSum(cpu_vars[i] * cpus.iloc[i]["Score"] for i in range(len(cpus))) >= game_data['CPU'],
    "CPU_Performance_Constraint",
)
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Score"] for i in range(len(gpus))) >= game_data['GPU'],
    "GPU_Performance_Constraint",
)

# RAM Capacity Constraint (ensure it has enough capacity for the game)
# Multiply by 1024 because capacity is in GB, no MB in RAM_Data.csv
problem += (
    lpSum(ram_vars[i] * rams.iloc[i]["Capacity (GB)"] * 1024 for i in range(len(rams))) >= game_data['memory'],  
    "RAM_Capacity_Constraint",
)

# Power constraint: Ensure PSU meets the recommended power for the selected GPU
problem += (
    lpSum(gpu_vars[i] * gpus.iloc[i]["Recommended Power"] for i in range(len(gpus))) <= 
    lpSum(psu_vars[i] * psus.iloc[i]["Wattage"] for i in range(len(psus))),
    "PSU_Power_Constraint",
)

# Ensure at most 2 RAM modules (either 1 or 2 identical modules) are selected
problem += lpSum(ram_vars) >= 1, "Min_One_RAM"
problem += lpSum(ram_vars) <= 2, "Max_Two_RAM"

# Only one CPU, GPU, and PSU can be selected
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(ram_vars) == 1, "Select_One_RAM"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"

# Solve the problem
problem.solve()

# Output the results
selected_cpu = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Name"].values[0]
selected_gpu = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Name"].values[0]
selected_psu = psus.iloc[[i for i in range(len(psus)) if psu_vars[i].value() == 1]]["Name"].values[0]

# Find the RAM model where value > 0
selected_ram_model_index = [i for i in range(len(rams)) if ram_vars[i].value() > 0]
if selected_ram_model_index:
    selected_ram_model = rams.iloc[selected_ram_model_index[0]]["Model"]
    selected_ram_count = int(ram_vars[selected_ram_model_index[0]].value())
else:
    selected_ram_model = None
    selected_ram_count = 0

# Get the benchmarks for selected CPU and GPU
selected_cpu_score = cpus.iloc[[i for i in range(len(cpus)) if cpu_vars[i].value() == 1]]["Score"].values[0]
selected_gpu_score = gpus.iloc[[i for i in range(len(gpus)) if gpu_vars[i].value() == 1]]["Score"].values[0]

# Calculate total cost
total_cost = (
    sum(cpu_vars[i].value() * cpus.iloc[i]["Price"] for i in range(len(cpus))) +
    sum(gpu_vars[i].value() * gpus.iloc[i]["Price"] for i in range(len(gpus))) +
    sum(ram_vars[i].value() * rams.iloc[i]["Price"] for i in range(len(rams))) +
    sum(psu_vars[i].value() * psus.iloc[i]["Price"] for i in range(len(psus)))
)

# Calculate the total RAM capacity
total_ram_capacity = selected_ram_count * rams.iloc[selected_ram_model_index[0]]["Capacity (GB)"] * 1024

# Display the results
print(f"Required CPU Benchmark for {selected_game}: {game_data['CPU']}")
print(f"Required GPU Benchmark for {selected_game}: {game_data['GPU']}")
print(f"Required Memory for {selected_game}: {game_data['memory']} MB\n")

print(f"Selected CPU: {selected_cpu}")
print(f"CPU Benchmark Score: {selected_cpu_score}\n")
print(f"Selected GPU: {selected_gpu}")
print(f"GPU Benchmark Score: {selected_gpu_score}\n")

# Display the selected RAM model and its count (either 1 or 2 copies)
print(f"Selected RAM Model: {selected_ram_model}")
print(f"Selected RAM Count: {selected_ram_count}")
print(f"Total RAM Capacity: {total_ram_capacity} MB\n")
print(f"Selected PSU: {selected_psu}")
print(f"Total Cost: {total_cost}")


Required CPU Benchmark for Counter-Strike 2: 1300
Required GPU Benchmark for Counter-Strike 2: 4500
Required Memory for Counter-Strike 2: 8192 MB

Selected CPU: Ryzen 3 1200
CPU Benchmark Score: 6282

Selected GPU: Gigabyte GTX 1050 2GB OC
GPU Benchmark Score: 5031

Selected RAM Model: DDR4
Selected RAM Count: 1
Total RAM Capacity: 8192 MB

Selected PSU: Gigabyte P450B 
Total Cost: 618.0
